
<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```


### Resolución
El polinomio exhibido anteriormente puede ser representado por la siguiente factorización:
\begin{equation}
f(x)= (A^2 x^2 - 2A^2 x + 1) (x - 1)
\end{equation}

Por lo que la primera raíz correspondería a 1, y las siguientes son calculadas con las operaciones entregadas

En cuanto a las otras dos raices, son obtenidas mediante
\begin{equation}
\frac{A^2 + \sqrt{(A^4 - A^2)}}{A^2} 
\end{equation}
y
\begin{equation}
\frac{A^2 - \sqrt{(A^4 - A^2)}}{A^2} 
\end{equation}

o escritas de la forma en que se procesaran mas abajo:

\begin{equation}
1 + \frac{\sqrt{(A^2 - 1)}}{A} 
\end{equation}

\begin{equation}
1 - \frac{\sqrt{(A^2 - 1)}}{A} 
\end{equation}


In [ ]:
import bitstring as bs
import numpy as np

"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant,bits_exp):

    if (bits_mant >= 2) and (bits_mant <=52) and (bits_exp >= 2) and (bits_exp <=11):
        
        #CITA 1
        b = bs.pack('>d',x)
        signo = b[0]
        exponente = b[1:bits_exp]
        mantisa = b[bits_exp:] 
        rep = 0
        NaN = False

        #Caso en que el exponente solo tenga 0's
        exp_0 = True
        
        #Caso en que el exponente solo tenga 1's 
        exp_1 = True

        #Caso que la mantisa solo tenga 0's
        mant_0 = True

        #Caso que la mantisa solo tenga 1's
        mant_1 = True

        #Verificar cada numero que compone el exponente
        for i in range(0,len(exponente)):
            if exponente[i] == 0:
                exp_1 = False
            elif exponente[i] == 1:
                exp_0 = False

        #Verificar cada numero que compone la mantisa
        for i in range(bits_mant,len(mantisa)):
            if mantisa[i] == 1:
                mant_0 = False
            elif mantisa[i] == 0:
                mant_1 = False

        #Caso signo = 0; solo 0's en mantisa y solo 1's en exponente
        if (signo == False) and (mant_0 == True) and (exp_1 == True):
            rep = float('inf')
            result = rep
            return result

        #Caso signo 1;solo 0's en matisa y solo 1's en exponente
        elif (signo == True ) and (mant_0 == True) and (exp_1 == True):
            rep = float('-inf')
            result = rep
            return result

        #Caso que haya solo 1's en exponente y mantisa
        elif (mant_1 == True) and (exp_1 == True):
            NaN = True
            result = np.nan
            return result

        elif (exp_0 == True):
            signo = False
        else:
            
            #Caso en que el bit en la posición bits_mant es 0 y
            #hay que redondear hacia abajo los que siguen
            if mantisa[bits_mant] == 0:
                for i in range(bits_mant, len(mantisa)):
                    mantisa[i] = 0

            #Caso en que el bit en la posición bits_mant es 1 y
            #Se observan diversos sub-casos
            elif mantisa[bits_mant] == 1:
                ceros = True

                #For que comprueba si hay solo 0's después del bit 53
                for i in range(bits_mant + 1, len(mantisa)):
                    if mantisa[i] != 0:
                        ceros = False

                #El bit 53 es 1, todos los bits después de este son 0 y el bit 52 es 0: truncar después del bit 52
                if (mantisa[bits_mant - 1] == 0) and (ceros == True):
                    for i in range(bits_mant, len(mantisa)):
                        mantisa[i] = 0

                #El bit 53 es 1, todos los bits después de este son 0 y el bit 52 es 1: sumar 1 al bit 52
                elif (mantisa[bits_mant - 1] == 1) and (ceros == True):
                    lleno = True

                    #Verifica si en la mantisa está lleno de 1's. Si solo hay 1's se generaría overflow y por tanto, no se aproxima 
                    for i in range(12, bits_mant + 1):
                        if mantisa[i] == 0:
                            lleno = False

                    # Si la mantisa no esta llena de 1's antes del bit 52, entonces se le suma 1 a cada 1 hasta que se encuentre un 0, el cual se cambia por 1
                    if lleno == False:
                        c = 1
                        while(c < bits_mant):
                            if mantisa[bits_mant-c] == 1:
                                mantisa[bits_mant-c] = 0
                            elif mantisa[bits_mant-c] == 0:
                                mantisa[bits_mant-c] = 1
                                break
                            c+= 1

                #El bit 53 es 1 y hay bits después de este que no son 0: agregar 1 al bit 52
                elif (ceros == False):
                    if (mantisa[bits_mant - 1] == 1):
                        lleno = True
                        for i in range(12, bits_mant + 1):
                            if mantisa[i] == 0:
                                lleno = False
                        if lleno == False:
                            c=1
                            while(c < bits_mant):
                                if mantisa[bits_mant-c] == 1:
                                    mantisa[bits_mant-c] = 0
                                elif mantisa[bits_mant-c] == 0:
                                    mantisa[bits_mant-c] = 1
                                    break
                                c+= 1
                    else:
                        mantisa[bits_mant-1] = 1
        b[0] = signo
        b[1:bits_exp] = exponente
        b[bits_exp:] = mantisa
        result = b.float
        return  result


"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    #Simplemente se pasan ambos valores a la nueva representación y se suman
    
    valor1=float(f_new_rep(n1,bits_mant,bits_exp))
    valor2=float(f_new_rep(n2,bits_mant,bits_exp))
    result=float(f_new_rep(valor1+valor2,bits_mant,bits_exp))
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    #Simplemente se pasan ambos valores a la nueva representación y se multiplican
    
    valor1=float(f_new_rep(n1,bits_mant,bits_exp))
    valor2=float(f_new_rep(n2,bits_mant,bits_exp))
    result=float(f_new_rep(valor1*valor2,bits_mant,bits_exp))
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    #Simplemente se pasan ambos valores a la neva representación y se dividen
    
    valor1=float( f_new_rep(n1,bits_mant,bits_exp))
    valor2=float( f_new_rep(n2,bits_mant,bits_exp))
    result=float(f_new_rep(valor1/valor2,bits_mant,bits_exp))  
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    #El polinomio presentado puede representarse de la siguiente forma (obtenida por división sintética)
    
    #Primera raíz encontrada de antemano
    x_roots = [1]
    
    A_cuadrado = f_multiplicacion(A,A,bits_mant,bits_exp)
    A_1 = f_suma(A_cuadrado,-1,bits_mant,bits_exp)
    raiz = f_new_rep(np.sqrt(A_1),bits_mant,bits_exp)
    Div_A = f_division(raiz,A,bits_mant,bits_exp)
    root2 = f_suma(1,Div_A,bits_mant,bits_exp)
    root3 = f_suma(1,-Div_A,bits_mant,bits_exp)
    
    x_roots.append(root2)
    x_roots.append(root3)
    x_roots.sort()
    return x_roots
    


### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\log(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


#### Desarrollo

Ya que se solicita que las sustancias sean extraídas a temperatura 0, se tendría el caso de \begin{equation} T(t,\alpha,\beta) = 0 \end{equation} por ende, se insta a la búsqueda de raíces.

El método escogido para esta ocasión fue Newton-Raphson, debido a que es trivial el cálculo de la derivada de la función Temperatura.

In [ ]:
import numpy as np

#Se escogió el initial guess 23.5 debido a que era un punto que en general se mantenía fijo
#a pesar de las modificaciones a alfa y beta. (Respaldado por la gráfica de la función
# Aún así utilicé también a modo de prueba el x0 = 26 y escogí el mejor entre ambos para optimizar el resultado.


#Función que aplica el método de Newton con inital guess de 23.5 ( Aunque si se pasa otro t, también responde)
def newton_raphson(a,b, t = 23.5 ,TOL = 1e-10):

    #resultado mantiene la estructura xi+1 = xi - f(xi)/f'(xi) del método de newton
    resultado = t - t*(a*np.cos(t) + b*np.sin(np.log(t))) / (b*np.cos(np.log(t)) - a*t*np.sin(t))
    
    #Condición de término en base al error absoluto
    if abs(t-resultado) < TOL:
        return resultado
    
    #Llamado recursivo del método de newton
    else:
        return newton_raphson(a,b,resultado)

"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""
def raiz_de_oreman(alpha, beta, tol = 1e-10):
    
    #Usando initial guess de 23.5
    root1 = newton_raphson(alpha,beta,t=23.5)
    
    #Usando initial guess de 26
    root2 = newton_raphson(alpha,beta,t=26)
    
    #Escoger la mejor raiz
    if abs(root1-25) > abs(root2-25):
        root = root2
    else:
        root = root1
    
    return root


# Referencias

Pregunta 1 : Manera de descomponer un número en mantisa y exponente adquirida de "03_floating_point_arithmetic.ipynb"

Url: https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1v2/02_floating_point_arithmetic.ipynb